# Capitulo 10 - Hackeando e fatiando sequências

## Vector tomada #1: compatível com Vector2d

In [ ]:
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr( self._components)
    components = components[components.find('['):-1]
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [ ]:
Vector([3.1, 4.2])
Vector((3, 4, 5))
Vector (range(10))

## Protocolos e duck typing

In [ ]:
# Código do exemplo 1.1 reproduzido aqui por conveniência

import collections

Card = collections.namedtuple('Card' , ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()
  
  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits
    for rank in self.ranks]
  
  def __len__(self):
    return len(self._cards)
  
  def __getitem__(self, position):
    return self._cards[position]

## Vector tomada #2: uma sequência que permite fatiamento

In [ ]:
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr( self._components)
    components = components[components.find('['):-1]
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    return self._components[index]

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [ ]:
v1 = Vector([3, 4, 5])
len(v1)
v1[0], v1[-1]
v7 = Vector(range(7))
v7[1:4]

## Como funciona o fatiamento

In [ ]:
# Observando o comportamento de getitem e de fatias

class MySeq:
  def __getitem__(self, index):
    return index

s = MySeq()
s[1]
s[1:4]
s[1:4:2]
slice(1, 4, 2)
s[1:4:2, 9]
s[1:4:2, 7:9]

In [ ]:
# Inspecionando os atributos da classe slice
dir(slice)
slice(None, 10, 2).indices(5)
slice(-3, None, None).indices(5)

## Um `_getitem_` que considera fatias

In [ ]:
from array import array
import reprlib
import math
import numbers

class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr( self._components)
    components = components[components.find('['):-1]
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__( self, index):
    cls = type(self)
    
    if isinstance(index, slice):
      return cls(self._components[index])
    elif isinstance(index, numbers.Integral):
      return self._components[index]
    else:
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [ ]:
v7 = Vector(range(7))
v7[-1]
v7[1:4]
v7[-1:]
v7[1,2]

## Vector tomada #3: acesso dinâmico a atributos

In [38]:
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'
  shortcut_names = 'xyzt'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr( self._components)
    components = components[components.find('['):-1]
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    return self._components[index]

  def __getattr__( self, name):
    cls = type(self)
    
    if len(name) == 1:
      pos = cls.shortcut_names.find(name)
      if 0 <= pos < len(self._components):
        return self._components[pos]
    
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))

  def __setattr__(self, name, value):
    cls = type(self)
    
    if len(name) == 1:
      if name in cls.shortcut_names:
        error = 'readonly attribute {attr_name!r}'
    elif name.islower():
      error = "can't set attributes 'a' to 'z' in {cls_name!r}"
    else:
      error = ""
    
    if error:
      msg = error.format(cls_name=cls.__name__, attr_name=name)
      raise AttributeError(msg)
    super().__setattr__(name, value)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [37]:
v = Vector(range(5))
v
v.x
v.x = 10
v.x
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

## Vector tomada #4: hashing e um `==` mais rápido

In [43]:
import functools

2 * 3 * 4 * 5
functools.reduce(lambda a,b: a*b, range(1, 6))

120

In [46]:
# Três maneiras de calcular o xor acumulado de inteiros de 0 a 5

n = 0
for i in range(1, 6):
  n ^= i
n

import functools
functools.reduce(lambda a, b: a^b, range(6))

import operator
functools.reduce(operator.xor, range(6))

1

In [48]:
from array import array
import reprlib
import math
import functools
import operator

class Vector:
  typecode = 'd'
  shortcut_names = 'xyzt'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr( self._components)
    components = components[components.find('['):-1]
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))
  
  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    return self._components[index]

  def __getattr__( self, name):
    cls = type(self)
    
    if len(name) == 1:
      pos = cls.shortcut_names.find(name)
      if 0 <= pos < len(self._components):
        return self._components[pos]
    
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))

  def __setattr__(self, name, value):
    cls = type(self)
    
    if len(name) == 1:
      if name in cls.shortcut_names:
        error = 'readonly attribute {attr_name!r}'
    elif name.islower():
      error = "can't set attributes 'a' to 'z' in {cls_name!r}"
    else:
      error = ""
    
    if error:
      msg = error.format(cls_name=cls.__name__, attr_name=name)
      raise AttributeError(msg)
    super().__setattr__(name, value)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

## O fantástico `zip`

In [52]:
# A função embutida zip em funcionamento

from itertools import zip_longest

zip(range(3), 'ABC')
list(zip(range(3), 'ABC'))
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

## Vector tomada #5: formatação

In [55]:
from array import array
import reprlib
import math
import functools
import operator
import itertools

class Vector:
  typecode = 'd'
  shortcut_names = 'xyzt'
  
  def __init__(self, components):
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)
  
  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(self._components))
  
  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))
  
  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))
  
  def __bool__(self):
    return bool(abs(self))
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    cls = type(self)
    if isinstance(index, slice):
      return cls(self._components[index])
    elif isinstance(index, numbers.Integral):
      return self._components[index]
    else:
      msg = '{.__name__} indices must be integers'
      raise TypeError(msg.format(cls))

  def __getattr__( self, name):
    cls = type(self)
    
    if len(name) == 1:
      pos = cls.shortcut_names.find(name)
      if 0 <= pos < len(self._components):
        return self._components[pos]
    
      msg = '{.__name__!r} object has no attribute {!r}'
      raise AttributeError(msg.format(cls, name))

  def __setattr__(self, name, value):
    cls = type(self)
    
    if len(name) == 1:
      if name in cls.shortcut_names:
        error = 'readonly attribute {attr_name!r}'
    elif name.islower():
      error = "can't set attributes 'a' to 'z' in {cls_name!r}"
    else:
      error = ""
    
    if error:
      msg = error.format(cls_name=cls.__name__, attr_name=name)
      raise AttributeError(msg)
    super().__setattr__(name, value)

  def __format__(self, fmt_spec=""):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({))'
    
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  def angle(self, n):
    r = math.sqrt(sum(x * x for x in self[n:]))
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a
  
  def angles(self):
    return (self.angle(n) for n in range(1, len(self)))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)